In [50]:
import cv2 #To retrieve inputs from the webcam
import dlib #To detect keypoints from a face
import numpy as np
from scipy.spatial import distance as dist


In [51]:
detector = dlib.get_frontal_face_detector() #To detect one or more faces
predictor = dlib.shape_predictor('./shape_predictor_68_face_landmarks.dat') # detecting keypoints from faces
cap = cv2.VideoCapture(0) # To capture images from the webcam

In [52]:
THRESH = 0.2

### Draw rectangle around eyes

In [53]:
def draw_rectangle(image, color):
    eye_cascade = cv2.CascadeClassifier('haarcascade_eye.xml')
    eyes = eye_cascade.detectMultiScale(image, scaleFactor = 1.2,
                                        minNeighbors = 4)
    for (x,y,w,h) in eyes:
                cv2.rectangle(image,(x,y),(x+w,y+h),color,5)


In [54]:
def eye_ear(eye):
	# compute the euclidean distances between the two sets of
	# vertical eye landmarks (x, y)-coordinates
	A = dist.euclidean(eye[1], eye[5])
	B = dist.euclidean(eye[2], eye[4])
	# compute the euclidean distance between the horizontal
	# eye landmark (x, y)-coordinates
	C = dist.euclidean(eye[0], eye[3])
	# compute the eye aspect ratio
	ear = (A + B) / (2.0 * C)
	# return the eye aspect ratio
	return ear

### Check if the eye is closed or opened by calculating the ear of each eye

In [55]:
def check_ear(ear,treshold):
    if ear < treshold:
            draw_rectangle(image, (0,0,255))
            cv2.putText(image, "closed eyes, WARNING!!",(50, 50),cv2.FONT_HERSHEY_SIMPLEX,
                            1,(0, 0, 255),2, cv2.LINE_AA ) 
    else:
            draw_rectangle(image, (0,255,255))
            cv2.putText(image, 'opened eyes',(50, 50),cv2.FONT_HERSHEY_SIMPLEX,
                        1,(255, 0, 0),2, cv2.LINE_AA )
          
    

### Display eye landmarks

In [56]:
def display_eye_landmarks(eye):
    for i, (x, y) in enumerate(eye):
            cv2.circle(image, (x, y), 1, (255, 0, 255), -1)

In [57]:
while True:
    # Capture the image from the webcam
    ret,image = cap.read()

    # Convert the image color to grayscale
    
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Detect the face
    
    rects = detector(gray, 1)
    
    # Detect landmarks for each face
    
    for rect in rects:
        
        # Get the landmark points
        shape = predictor(gray, rect)
        
	    # Convert it to the NumPy Array
        shape_np = np.zeros((68, 2), dtype="int")

        for i in range(0, 68):
            shape_np[i] = (shape.part(i).x, shape.part(i).y)

        shape = shape_np
        
        #Define the left eye landmarks
        
        left_eye = np.zeros((6, 2), dtype="int")
        for j in range(0,6):
            left_eye[j] = shape[42 + j]

        #define the right eye landmarks
        
        right_eye = np.zeros((6, 2), dtype="int")
        for j in range(0,6):
            right_eye[j] = shape[36 + j]
            
        # Display the landmarks
        
        #display_eye_landmarks(left_eye)

        #display_eye_landmarks(right_eye)
       
        #Calculate the ear
        
        right_ear = eye_ear(right_eye)
        left_ear = eye_ear(left_eye)
        
        ear = (left_ear + right_ear) / 2.0
        
        # check to see if the eye ear is below the threshold
        check_ear(ear,THRESH)
          
    # Display the image
    cv2.imshow('Landmark Detection', image)

    # Press the escape button to terminate the code
    if cv2.waitKey(10) == 27:
        break

cap.release()

KeyboardInterrupt: 